<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" style="float:right; max-width: 250px; display: inline" alt="IMT"/> </a>
</center>

## Chatbot: catégorisations et prévisions des réponses aux questions

# Text Mining et Catégorisation de Produits en <a href="https://www.python.org/"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f8/Python_logo_and_wordmark.svg/390px-Python_logo_and_wordmark.svg.png" style="max-width: 120px; display: inline" alt="R"/></a> avec <a href="http://scikit-learn.org/stable/#"><img src="http://scikit-learn.org/stable/_static/scikit-learn-logo-small.png" style="max-width: 100px; display: inline" alt="R"/></a>

## Introduction

Il s'agit de prévoir la réponse à une question à partir de son sujet. Seule la catégorie principale (1er niveau) est prédite mais nous pourrons essayer d'affiner par la suite. L'objectif est plutôt de comparer les performances des méthodes et technologies en fonction de la taille de la base d'apprentissage ainsi que d'illustrer sur un exemple complexe le prétraitement de données textuelles. La stratégie de sous ou sur échantillonnage des catégories qui permet d'améliorer la prévision va être mise en oeuvre. Nous allons essayer différentes techniques d'échantillonnage comme le regroupement de plusieurs catégories en fonction de leur points communs ou selon leur taille puis nous essayerons de mettre en place un mélange de upsampling et de downsampling.
* Nous allons réduire l'échantillon réduit en le séparant en 2 parties: apprentissage et validation. 
* Les données textuelles sont  nettoyées, racinisées, vectorisées avant modélisation.
* Trois modélisations sont estimées: logistique, arbre, forêt aléatoire.
* Optimiser l'erreur en faisant varier différents paramètres: types et paramètres de vectorisation (TF-IDF), paramètres de la régression logistique (pénalisation l1) et de la forêt aléatoire (nombre d'arbres et nombre de variables aléatoire).

Exécuter finalement le code pour différentes tailles (paramètre  `tauxTot` ci-dessous) de l'échantillon d'apprentissage et comparer les qualités de prévision obtenues. 

In [2]:
#Importation des librairies utilisées
import unicodedata 
import time
import pandas as pd
import numpy as np
import random
import nltk
import collections
import itertools
import csv
import warnings

from sklearn.cross_validation import train_test_split

C:\Users\ETIENNE\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt

## 1. Importation des données
Définition du répertoir de travail, des noms des différents fichiers utilisés et des variables globales.


In [4]:
# Répertoire de travail
DATA_DIR = "C:/Users/ETIENNE/Documents/Work/INSA/4A/Projets 4gmm 2018/"

# Nom des fichiers
training_reduit_path = DATA_DIR + "INSA_wefight_data_clean.csv"
# Variable Globale
HEADER_TEST = ['Question','Intent','BlockId', 'Action']
HEADER_TRAIN =['Question','Intent','BlockId', 'Action']

In [5]:
## Si nécessaire (première exécution) chargement de nltk, librairie pour la suppression 
## des mots d'arrêt et la racinisation
## nltk.download()

   ### Read & Split Dataset
   Fonction permettant de lire le fichier d'apprentissage et de créer deux DataFrame Pandas, un pour l'apprentissage, l'autre pour la validation.
   La première méthode créée un DataFrame en lisant entièrement le fichier. Puis elle scinde le DataFrame en deux  grâce à la fonction dédiée de sklearn. 

In [6]:
def split_dataset(input_path, nb_line, tauxValid,columns):
    time_start = time.time()
    data_all = pd.read_csv(input_path,sep=",",names=columns,nrows=nb_line) #cree data frame
    data_all = data_all.fillna("") #remplace les na par " "
    data_train, data_valid = train_test_split(data_all, test_size = tauxValid) # Split arrays or matrices into random train and test subsets
    time_end = time.time()
    print("Split Takes %d s" %(time_end-time_start))
    return data_train, data_valid

nb_line=20000  # part totale extraite du fichier initial ici déjà réduit
tauxValid=0.10 # part totale extraite du fichier initial ici déjà réduit
data_train, data_valid = split_dataset(training_reduit_path, nb_line, tauxValid, HEADER_TRAIN)
# Cette ligne permet de visualiser les 5 premières lignes de la DataFrame 
data_train.head(5)

Split Takes 0 s


,Question,Intent,BlockId,Action
1541,j'ai du mal à évaluer ma douleur,#2-55_QVDP_Douleur,59843bb8e4b03f0d1304835d,wiki_cancer
2573,à quoi ser l'hormonothérapie?,#6-24_TRTEINS_hormonotherapie,59632c41e4b0a226d067cc6d,wiki_cancer
1945,quel sport privilégier,#2-79_QVDP_SportQuel,59895d38e4b03f0d2cdd37b1,wiki_cancer
2294,c'est quoi une chimio ambulatoire ?,#6-12_TRTEINS_ChimioAmbulatoire,59632c41e4b0a226d067ccb9,wiki_cancer
837,Que manger pendant mon traitement,#2-130_QVDP_Alimentation,5991c543e4b0b2045b0c568f,wiki_cancer


## 2. Nettoyage des données
Afin de limiter la dimension de l'espace des variables ou *features*, tout en conservant les informations essentielles, il est nécessaire de nettoyer les données en appliquant plusieurs étapes:
* Chaque mot est écrit en minuscule.
* Les termes numériques, de ponctuation et autres symboles sont supprimés.
* 155 mots-courants, et donc non informatifs, de la langue française sont supprimés (STOPWORDS). Ex: le, la, du, alors, etc...
* Chaque mot est "racinisé", via la fonction `STEMMER.stem` de la librairie nltk. La racinisation transforme un mot en son radical ou sa racine. Par exemple, les mots: cheval, chevaux, chevalier, chevalerie, chevaucher sont tous remplacés par "cheva".

### Importation des librairies et fichier pour le nettoyage des données.

In [7]:
# Librairies 
from bs4 import BeautifulSoup #Nettoyage d'HTML
import re # Regex
import nltk # Nettoyage des données

## listes de mots à supprimer dans la description des produits
## Depuis NLTK
nltk_stopwords = nltk.corpus.stopwords.words('french') 
## Depuis Un fichier externe.
lucene_stopwords = [unicode(w, "utf-8") for w in open(DATA_DIR+"lucene_stopwords.txt").read().split(",")] #En local

## Union des deux fichiers de stopwords 
stopwords = list(set(nltk_stopwords).union(set(lucene_stopwords)))

## Fonction de setmming de stemming permettant la racinisation
stemmer=nltk.stem.SnowballStemmer('french')

### Fonction de nettoyage de texte
Fonction qui prend en intrée un texte et retourne le texte nettoyé en appliquant successivement les étapes suivantes: Nettoyage des données HTML, conversion en texte minuscule, encodage uniforme, suppression des caractéres non alpha numérique (ponctuations), suppression des stopwords, racinisation de chaque mot individuellement.

In [8]:
# remarque
#'a b c'.split()
#str.split('a b c')
# both return ['a', 'b', 'c']

In [9]:
# Fonction clean générale
def clean_txt(txt):
    ### remove html stuff
    txt = BeautifulSoup(txt,"html.parser",from_encoding='utf-8').get_text() #nettoyage donnee html
    ### lower case
    txt = txt.lower()
    ### special escaping character '...'
    txt = txt.replace(u'\u2026','.')
    txt = txt.replace(u'\u00a0',' ')
    ### remove accent btw
    txt = unicodedata.normalize('NFD', txt).encode('ascii', 'ignore')
    ###txt = unidecode(txt)
    ### remove non alphanumeric char
    txt = re.sub('[^a-z_]', ' ', txt)
    ### remove french stop words
    tokens = [w for w in txt.split() if (len(w)>2) and (w not in stopwords)]
    ### french stemming
    tokens = [stemmer.stem(token) for token in tokens]
    #Stemmers remove morphological affixes from words, leaving only the word stem
    ### tokens = stemmer.stemWords(tokens)
    return ' '.join(tokens)
    #join() returns a string in which the string elements of sequence have been joined by str separator.

def clean_marque(txt):
    txt = re.sub('[^a-zA-Z0-9]', '_', txt).lower()
    return txt

### Nettoyage des DataFrames
Applique le nettoyage sur toutes les lignes de la DataFrame

In [10]:
# fonction de nettoyage du fichier(stemming et liste de mots à supprimer)
def clean_df(input_data, column_names= ['Question','Intent','BlockId', 'Action']):
    #Test if columns entry match columns names of input data
    column_names_diff= set(column_names).difference(set(input_data.columns))
    #set.difference   new set with elements in column_names but not in input_data.columns
    
    if column_names_diff: #rentre dans la boucle si column_names différent zero
        # warning = exception
        warnings.warn("Column(s) '"+", ".join(list(column_names_diff)) +"' do(es) not match columns of input data", Warning)
        
    nb_line = input_data.shape[0]
    print("Start Clean %d lines" %nb_line)
    
    # Cleaning start for each columns
    time_start = time.time()
    clean_list=[]
    for column_name in column_names:
        column = input_data[column_name].values
        if column_name == "Question":
            array_clean = np.array(map(clean_txt,column))
            
        elif column_name == "Intent":
            array_clean = np.asarray(input_data['Intent']) #on recopie telle quelle la colonne intent  
            
        else:
            array_clean = np.array(map(clean_marque,column))
            #applies a function to all the items in an input_list
            #map(function_to_apply, list_of_inputs)
        clean_list.append(array_clean)
    time_end = time.time()
    print("Cleaning time: %d secondes"%(time_end-time_start))
    
    #Convert list to DataFrame
    array_clean = np.array(clean_list).T
    data_clean = pd.DataFrame(array_clean, columns = column_names)
    return data_clean

In [11]:
# Take approximately 2 minutes fors 100.000 rows
data_valid_clean = clean_df(data_valid)
data_train_clean = clean_df(data_train)

Start Clean 502 lines
Cleaning time: 0 secondes
Start Clean 4511 lines
Cleaning time: 3 secondes


Affiche les 5 premières lignes de la DataFrame d'apprentissage après nettoyage.

In [12]:
#info sur les données
data_train_clean.info()
data_train_clean.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4511 entries, 0 to 4510
Data columns (total 4 columns):
Question    4511 non-null object
Intent      4511 non-null object
BlockId     4511 non-null object
Action      4511 non-null object
dtypes: object(4)
memory usage: 141.0+ KB


,Question,Intent,BlockId,Action
0,mal evalu douleur,#2-55_QVDP_Douleur,59843bb8e4b03f0d1304835d,wiki_cancer
1,ser hormonotherap,#6-24_TRTEINS_hormonotherapie,59632c41e4b0a226d067cc6d,wiki_cancer
2,sport privilegi,#2-79_QVDP_SportQuel,59895d38e4b03f0d2cdd37b1,wiki_cancer
3,chimio ambulatoir,#6-12_TRTEINS_ChimioAmbulatoire,59632c41e4b0a226d067ccb9,wiki_cancer
4,mang trait,#2-130_QVDP_Alimentation,5991c543e4b0b2045b0c568f,wiki_cancer


In [13]:
data_train_clean

,Question,Intent,BlockId,Action
0,mal evalu douleur,#2-55_QVDP_Douleur,59843bb8e4b03f0d1304835d,wiki_cancer
1,ser hormonotherap,#6-24_TRTEINS_hormonotherapie,59632c41e4b0a226d067cc6d,wiki_cancer
2,sport privilegi,#2-79_QVDP_SportQuel,59895d38e4b03f0d2cdd37b1,wiki_cancer
3,chimio ambulatoir,#6-12_TRTEINS_ChimioAmbulatoire,59632c41e4b0a226d067ccb9,wiki_cancer
4,mang trait,#2-130_QVDP_Alimentation,5991c543e4b0b2045b0c568f,wiki_cancer
5,preven apparit aphte,#6-53_TRTEINS_Aphtes,59918dbae4b0feb28887a14b,wiki_cancer
6,syndrom yeux sec,#6-57_TRTEINS_Yeux_Secs,59918e4de4b0feb2888bc59e,wiki_cancer
7,canc pert poid,#6-96_TRTEINS_Perte_Poids,5993f2cae4b068eebe3131e2,wiki_cancer
8,peux utilis shampoing sec,#2-41_QVDP_Alopecie_Diminuer,59843588e4b03f0d12d83b08,wiki_cancer
9,conseil alimentair femm hormonotherap prend po...,#6-31_TRTEINS_Effetsecondaireshormonotherapie,59632c41e4b0a226d067cee4,wiki_cancer


In [14]:
#description des données
data_train_clean.describe()

,Question,Intent,BlockId,Action
count,4511,4511,4511,4511
unique,3806,144,132,19
top,rend,#6-49_TRTEINS_Peau,59918cf1e4b0feb28881fd9f,wiki_cancer
freq,20,197,197,4250


In [15]:
# comptage u nbre d'occurence de categorie puis comptage du nbre de categorie differente
print(data_train["Action"].value_counts())
print("nb Action differente",(data_train["Action"].value_counts()).shape)

wiki_cancer                                          4250
                                                       81
conversation_rappelRendezVous                          79
conversation_rappelRead                                31
conversation_rappelUpdate2                             18
wiki_cancer#                                           13
conversation_FichePatientRead                          10
conversation_FichePatientUpdate                         5
conversation_hist                                       4
conversation_FichePatientWrite:age                      4
conversation_rappelUpdate                               3
conversation_FichePatientWrite:doseTraitement           3
conversation_FichePatientWrite:typeTraitement           2
conversation_FichePatientWrite:rappels                  2
conversation_FichePatientWrite:newsletter               2
conversation_FichePatientWrite:recevoirTemoignage       1
conversation_FichePatientWrite:role                     1
conversation_F

In [16]:
# comptage u nbre d'occurence de categorie puis comptage du nbre de categorie differente
print(data_train["Intent"].value_counts())
print("nb intent differente",(data_train["Intent"].value_counts()).shape)

#6-49_TRTEINS_Peau                               197
#6-92_TRTEINS_Diarrhee                           129
#2-130_QVDP_Alimentation                         122
#6-97_TRTEINS_Nausees_Vomissements               121
#6-53_TRTEINS_Aphtes                             112
#6-90_TRTEINS_Mauvais_Gout                       105
#6-98_TRTEINS_EI_Frequents                       102
#2-36_QVDP_Alopecie_Pourquoi                      99
#9-2_Informations_cancer                          96
#6-60_TRTEINS_PAC                                 93
#2-64-0_QVDP_Fatigue                              87
#6-57_TRTEINS_Yeux_Secs                           86
#6-18_TRTEINS_Radiotherapie                       85
#2-55_QVDP_Douleur                                81
conversation_rappel_rendezvous                    79
#6-96_TRTEINS_Perte_Poids                         75
#6-1_TRTEINS_Chimiotherapie                       74
#2-45_QVDP_Alopecie_Perruque                      71
#2-96_QVDP_Social_Priseencharge               

In [17]:
#on enregistre le contenu de la colonne Question dans un fichier csv
#on fait ensuite un wordcloud sur toutes les donnees
description = data_train_clean["Question"]
description.to_csv('Question.csv', sep = ',')

In [18]:
# tri par catégorie, on a un data frame pour une catégorie
df = data_train_clean[data_train_clean['Intent'] == '#6-49_TRTEINS_Peau']
df

,Question,Intent,BlockId,Action
31,soin dermatolog,#6-49_TRTEINS_Peau,59918cf1e4b0feb28881fd9f,wiki_cancer
49,gel surgr,#6-49_TRTEINS_Peau,59918cf1e4b0feb28881fd9f,wiki_cancer
69,lav,#6-49_TRTEINS_Peau,59918cf1e4b0feb28881fd9f,wiki_cancer
261,malad peau,#6-49_TRTEINS_Peau,59918cf1e4b0feb28881fd9f,wiki_cancer
268,nettoyag peau,#6-49_TRTEINS_Peau,59918cf1e4b0feb28881fd9f,wiki_cancer
284,hello souffr beaucoup brulur scan crem fait rien,#6-49_TRTEINS_Peau,59918cf1e4b0feb28881fd9f,wiki_cancer
303,tach roug sein,#6-49_TRTEINS_Peau,59918cf1e4b0feb28881fd9f,wiki_cancer
306,trait peau pel,#6-49_TRTEINS_Peau,59918cf1e4b0feb28881fd9f,wiki_cancer
338,produit secheress,#6-49_TRTEINS_Peau,59918cf1e4b0feb28881fd9f,wiki_cancer
345,peau bouch,#6-49_TRTEINS_Peau,59918cf1e4b0feb28881fd9f,wiki_cancer


## 3 Construction des caractéristiques ou *features* (TF-IDF)¶
### Introduction
La vectorisation, c'est-à-dire la construction des caractéristiques à partir de la liste des mots se fait en 2 étapes:
* **Hashage**. Il permet de réduire l'espace des variables (taille du dictionnaire) en un nombre limité et fixé a priori `n_hash` de caractéristiques. Il repose sur la définition d'une fonction de hashage, $h$ qui à un indice $j$ défini dans l'espace des entiers naturels, renvoie un indice $i=h(j)$ dans dans l'espace réduit (1 à n_hash) des caractéristiques. Ainsi le poids de l'indice $i$, du nouvel espace, est l'association de tous les poids d'indice $j$ tels que $i=h(j)$ de l'espace originale. Ici, les poids sont associés d'après la méthode décrite par Weinberger et al. (2009).

N.B. $h$ n'est pas généré aléatoirement. Ainsi pour un même fichier d'apprentissage (ou de test) et pour un même entier n_hash, le résultat de la fonction de hashage est identique

* **TF-IDF**. Le TF-IDF permet de faire ressortir l'importance relative de chaque mot $m$ (ou couples de mots consécutifs) dans un texte-produit ou un descriptif $d$, par rapport à la liste entière des produits. La fonction $TF(m,d)$ compte le nombre d'occurences du mot $m$ dans le descriptif $d$. La fonction $IDF(m)$ mesure l'importance du terme dans l'ensemble des documents ou descriptifs en donnant plus de poids aux termes les moins fréquents car considérés comme les plus discriminants (motivation analogue à celle de la métrique du chi2 en anamlyse des correspondance). $IDF(m,l)=\log\frac{D}{f(m)}$ où $D$ est le nombre de documents, la taille de l'échantillon d'apprentissage, et $f(m)$ le nombre de documents ou descriptifs contenant le mot $m$. La nouvelle variable ou *features* est $V_m(l)=TF(m,l)\times IDF(m,l)$.

* Comme pour les transformations des variables quantitatives (centrage, réduction), la même transformation c'est-à-dire les mêmes pondérations, est calculée sur l'achantillon d'apprentissage et appliquée à celui de test.

In [24]:
#meta classe

categories = data_train_clean['Intent']
tailleCat = int(np.shape(categories)[0])
LabelCat = np.zeros(tailleCat)
for k in range(tailleCat):
    # la categorie #0 a le label 0 implicitement
    if categories[k][0:2] == 'co' : LabelCat[k] = 10 #conversation_rappel_rendezvous
    if categories[k][0:2] == '#1' : LabelCat[k] = 1
    if categories[k][0:2] == '#2' : LabelCat[k] = 2
    if categories[k][0:2] == '#3' : LabelCat[k] = 3
    if categories[k][0:2] == '#4' : LabelCat[k] = 4
    if categories[k][0:2] == '#5' : LabelCat[k] = 5
    if categories[k][0:2] == '#6' : LabelCat[k] = 6
    if categories[k][0:2] == '#7' : LabelCat[k] = 7
    if categories[k][0:2] == '#8' : LabelCat[k] = 8
    if categories[k][0:2] == '#9' : LabelCat[k] = 9

47


In [20]:
#data frame avec une colonne mettant un label suivant la meta categorie
dfLabelCat = pd.DataFrame(LabelCat)
meta_df = pd.concat([data_train_clean, dfLabelCat], axis=1)
meta_df.head()

,Question,Intent,BlockId,Action,0
0,mal evalu douleur,#2-55_QVDP_Douleur,59843bb8e4b03f0d1304835d,wiki_cancer,2.0
1,ser hormonotherap,#6-24_TRTEINS_hormonotherapie,59632c41e4b0a226d067cc6d,wiki_cancer,6.0
2,sport privilegi,#2-79_QVDP_SportQuel,59895d38e4b03f0d2cdd37b1,wiki_cancer,2.0
3,chimio ambulatoir,#6-12_TRTEINS_ChimioAmbulatoire,59632c41e4b0a226d067ccb9,wiki_cancer,6.0
4,mang trait,#2-130_QVDP_Alimentation,5991c543e4b0b2045b0c568f,wiki_cancer,2.0


In [27]:
#on stocke les questions pour chaque cle, chaque cle sera une categorie
dix = []
neuf = []
huit = []
sept = []
six = []
cinq = []
quatre = []
trois = []
deux = []
un = []
zero = []

for k in range(tailleCat):
    if meta_df[0][k] == 10.0 : dix.append(meta_df['Question'][k])
    if meta_df[0] [k] == 9.0 : neuf.append (meta_df['Question'][k])
    if meta_df[0][k] == 8.0 : huit.append (meta_df['Question'][k])
    if meta_df[0][k] == 7.0 : sept.append (meta_df['Question'][k])
    if meta_df[0][k] == 6.0 : six.append (meta_df['Question'][k])
    if meta_df[0][k] == 5.0 : cinq.append (meta_df['Question'][k])
    if meta_df[0][k] == 4.0 : quatre.append(meta_df['Question'][k])
    if meta_df[0][k] == 3.0 : trois.append (meta_df['Question'][k])
    if meta_df[0][k] == 2.0 : deux.append (meta_df['Question'][k])
    if meta_df[0][k] == 1.0 : un.append (meta_df['Question'][k])
    if meta_df[0][k] == 0.0 : zero.append (meta_df['Question'][k])
        
label = range(11)
questionLab = [zero,un,deux,trois,quatre,cinq,six,sept,huit,neuf,dix]

meta_dict = dict(zip(label,questionLab)) # creation du dictionnaire

In [28]:
meta_dict

{0: [u'rappel pris medic',
  u'derang',
  u'list rappel',
  u'met rappel soir',
  u'infos',
  u'age',
  u'envoi messag',
  u'matin encor oubl rappel pris nolvadex mal programm demand problem',
  u'dos trait',
  u'rappel heur prendr tamoxifen',
  u'besoin',
  u'rzppel',
  u'arret rappel',
  u'montr fich',
  u'aid bien faut arret foutr bien',
  u'modifi rappel',
  u'rappel prendr tamoxifen jour mid mois',
  u'reflech taguel',
  u'conner',
  u'rappel prendr trait',
  u'typ trait',
  u'',
  u'info',
  u'met rappel jour stp',
  u'veux surtout aid fais peux vivr alor ras bol demand macron',
  u'rappel',
  u'surtout recontact crois vecu survecu pir',
  u'infos',
  u'propos rappel',
  u'arret rappel',
  u'connard',
  u'alor canc col uterus',
  u'veux voir fich',
  u'rappel prendrem trait',
  u'laiss rappel',
  u'montr fich perso',
  u'mettr jour profil',
  u'rappel medic',
  u'montr med rappel',
  u'age',
  u'voir crant',
  u'inform personnel',
  u'',
  u'montr rappel',
  u'veux voir fich',
  